In [64]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer
breast_data = load_breast_cancer()
data, target = breast_data['data'],  breast_data['target']
columns = breast_data.feature_names

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      stratify=target,
                                                                      random_state=42,
                                                                      test_size=0.2)

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(StandardScaler(),
                     SVC(random_state=42))
param_range = [0.0001, 0.001, 0.01, 0.1, 10, 100, 1000]
param_grid = [{'svc__C':param_range,
               'svc__kernel':['linear']},
              {'svc__C':param_range,
               'svc__gamma':param_range,
               'svc__kernel':['rdf']}]

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
hs = HalvingRandomSearchCV(pipe,
                           param_distributions=param_grid,
                           n_candidates='exhaust',
                           factor=1.5,
                           random_state=42,
                           refit=True,
                           resource='n_samples',
                           n_jobs=-1)
hs = hs.fit(train_input, train_target)
print(hs.best_score_)
print(hs.best_params_)
print(hs.score(train_input, train_target))

0.973529411764706
{'svc__kernel': 'linear', 'svc__C': 0.1}
0.9846153846153847


In [68]:
from sklearn.model_selection import GridSearchCV, cross_validate
gs = GridSearchCV(estimator=pipe,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2)
scores = cross_validate(gs, train_input, train_target,
                        scoring='accuracy', cv=5)['test_score']
print(np.mean(scores))

0.9780219780219781


In [71]:
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                  param_grid=[{'max_depth':[1, 2, 3, 4, 5, 6, 7, None]}],
                  scoring='accuracy',
                  cv=2)
scores = cross_validate(gs, train_input, train_target,
                        scoring='accuracy', cv=5)['test_score']
print(np.mean(scores))

0.9186813186813186
